In [11]:
import os
import shutil
import random
from pathlib import Path

# إعداد المسارات
images_folder = Path(r"C:\Users\abdul\Desktop\YoloV8_New_Attempt\datasets\images_all\Train")
labels_folder = Path(r"C:\Users\abdul\Desktop\YoloV8_New_Attempt\output_labels")
base_output = Path(r"C:/Users/abdul/Desktop/YoloV8_New_Attempt/datasets/football_pitch")

# إنشاء مجلدات الإخراج
for split in ["train", "val", "test"]:
    os.makedirs(base_output / "images" / split, exist_ok=True)
    os.makedirs(base_output / "labels" / split, exist_ok=True)

# جمع كل الصور
image_files = list(images_folder.glob("*.jpg")) + list(images_folder.glob("*.png"))
random.seed(42)
random.shuffle(image_files)

# التقسيم: 80% train, 10% val, 10% test
n_total = len(image_files)
n_train = int(0.8 * n_total)
n_val = int(0.1 * n_total)
n_test = n_total - n_train - n_val

splits = {
    "train": image_files[:n_train],
    "val": image_files[n_train:n_train + n_val],
    "test": image_files[n_train + n_val:]
}

# نقل الصور وملفات .txt إلى أماكنها المناسبة
for split, files in splits.items():
    for img_path in files:
        label_path = labels_folder / (img_path.stem + ".txt")
        if label_path.exists():
            shutil.copy(img_path, base_output / "images" / split / img_path.name)
            shutil.copy(label_path, base_output / "labels" / split / label_path.name)

# توليد ملف data.yaml
data_yaml = f"""path: {base_output.as_posix()}
train: images/train
val: images/val
test: images/test

nc: 1
names: ['pitch_layout']
kpt_shape: [14, 3]
flip_idx: [2, 1, 0, 9, 10, 8, 6, 7, 5, 3, 4, 13, 12, 11]
"""

with open(base_output / "football_pitch_data.yaml", "w", encoding="utf-8") as f:
    f.write(data_yaml)

base_output.as_posix()


'C:/Users/abdul/Desktop/YoloV8_New_Attempt/datasets/football_pitch'

In [12]:
from pathlib import Path

# المسارات الجديدة حسب كلامك
image_dir = Path(r"C:\Users\abdul\Desktop\YoloV8_New_Attempt\datasets\images_all\Train")
label_dir = Path(r"C:\Users\abdul\Desktop\YoloV8_New_Attempt\output_labels")

# نقرأ أسماء الصور والليبلات
image_files = {img.name.rsplit(".", 1)[0]: img for img in image_dir.glob("*") if img.suffix.lower() in [".jpg", ".jpeg", ".png"]}
label_files = {lbl.name.rsplit(".", 1)[0]: lbl for lbl in label_dir.glob("*.txt")}

# المفاتيح المتطابقة
matched_keys = sorted(set(image_files) & set(label_files))

# نطبع حالة التناسق
{
    "عدد الصور": len(image_files),
    "عدد الليبلات": len(label_files),
    "عدد الملفات المتطابقة": len(matched_keys),
    "صور بدون ليبل": sorted(set(image_files) - set(label_files))[:5],
    "ليبلات بدون صور": sorted(set(label_files) - set(image_files))[:5],
}


{'عدد الصور': 466,
 'عدد الليبلات': 579,
 'عدد الملفات المتطابقة': 462,
 'صور بدون ليبل': ['frame_009054',
  'frame_009074',
  'frame_009114',
  'frame_009134'],
 'ليبلات بدون صور': ['frame_000305',
  'frame_000310',
  'frame_000315',
  'frame_000320',
  'frame_000325']}

In [ ]:
import os, shutil, random
from math import floor
from pathlib import Path

image_dir = Path(r"datasets\images\Train")  
label_dir = Path(r"output_labels")  

output_base = Path(r"datasets\football_pitch") 


for split in ["train", "val", "test"]:
    (output_base / "images" / split).mkdir(parents=True, exist_ok=True)
    (output_base / "labels" / split).mkdir(parents=True, exist_ok=True)


image_files = {img.stem: img for img in image_dir.glob("*") if img.suffix.lower() in [".png", ".jpg", ".jpeg"]}
label_files = {lbl.stem: lbl for lbl in label_dir.glob("*.txt")}
matched_keys = sorted(set(image_files) & set(label_files))

total = len(matched_keys)  
train_count = floor(total * 0.8)  
val_count = floor(total * 0.1)    
test_count = total - train_count - val_count
random.shuffle(matched_keys)

split_map = {
    'train': matched_keys[:train_count],
    'val': matched_keys[train_count:train_count + val_count],
    'test': matched_keys[train_count + val_count:]
}

for split, keys in split_map.items():
    for k in keys:
        shutil.copy(image_files[k], output_base / "images" / split / image_files[k].name)
        shutil.copy(label_files[k], output_base / "labels" / split / (k + ".txt"))

{split: len(keys) for split, keys in split_map.items()}


{'train': 369, 'val': 46, 'test': 47}

In [ ]:
import torch
from ultralytics import YOLO
import time
import datetime
import collections 

def train_pitch_pose_model():
    """YOLOv8-Pose augmentation."""
    start_time = time.time()
    print(f"Training started at: {datetime.datetime.now()}")

    # مسار ملف data.yaml
    DATA_YAML_PATH = r"data.yaml" 

    # النموذج
    MODEL_WEIGHTS = "yolov8m-pose.pt"

    # إعدادات التدريب
    EPOCHS = 250
    IMG_SIZE = 960 
    BATCH_SIZE = 8
    DEVICE = 0
    OPTIMIZER = "AdamW"
    LR = 1e-3
    PATIENCE = 50
    WARMUP_EPOCHS = 5
    WEIGHT_DECAY = 0.0005
    LRF = 0.01

    # Augmentations (فقط اللونية مفعلة)
    MOSAIC = 0.0         # <-- تعطيل
    MIXUP = 0.0          # <-- تعطيل (كان معطلاً)
    DEGREES = 0.0        # <-- تعطيل
    TRANSLATE = 0.0      # <-- تعطيل
    SCALE = 0.0          # <-- تعطيل
    SHEAR = 0.0          # <-- تعطيل
    PERSPECTIVE = 0.0    # <-- تعطيل
    FLIPUD = 0.0         # <-- تعطيل (كان معطلاً)
    FLIPLR = 0.0         # <-- تعطيل (مهم جدًا لتجنب تأثير flip_idx الخاطئ)
    # --- الإبقاء على اللونية ---
    HSV_H = 0.015
    HSV_S = 0.7
    HSV_V = 0.4

    PROJECT_NAME = "Pitch_Pose_Project_ColorAugOnly" # تغيير اسم المشروع لتمييزه
    RUN_NAME = f"pose_e{EPOCHS}_img{IMG_SIZE}_clrAug_{int(time.time())}"

    # التحقق من وجود CUDA
    if not torch.cuda.is_available():
        print("CUDA is not available. Training on CPU (بطيء جدًا).")
        DEVICE = "cpu" # تغيير الجهاز إلى CPU إذا لم تكن CUDA متاحة

    # إنشاء نموذج YOLO
    try:
        model = YOLO(MODEL_WEIGHTS)
        print(f"Model {MODEL_WEIGHTS} loaded successfully.")
    except Exception as e:
        print(f"Error loading model {MODEL_WEIGHTS}: {e}")
        return # الخروج إذا لم يتم تحميل النموذج

    # تنفيذ التدريب
    try:
        print("Starting model training...")
        results = model.train(
            data=DATA_YAML_PATH,
            epochs=EPOCHS,
            imgsz=IMG_SIZE,
            batch=BATCH_SIZE,
            device=DEVICE,
            optimizer=OPTIMIZER,
            lr0=LR,
            patience=PATIENCE,
            warmup_epochs=WARMUP_EPOCHS,
            weight_decay=WEIGHT_DECAY,
            lrf=LRF,
            # --- تمرير قيم Augmentation المعدلة ---
            mosaic=MOSAIC,
            mixup=MIXUP,
            degrees=DEGREES,
            translate=TRANSLATE,
            scale=SCALE,
            shear=SHEAR,
            perspective=PERSPECTIVE,
            flipud=FLIPUD,
            fliplr=FLIPLR, # <-- القيمة أصبحت 0.0
            hsv_h=HSV_H,
            hsv_s=HSV_S,
            hsv_v=HSV_V,
            # إعدادات أخرى
            project=PROJECT_NAME,
            name=RUN_NAME,
            cache="ram",
            workers=0,
            exist_ok=True,
            save=True,
            save_period=-1,
            val=True,
            plots=True
        )
        print("Training function finished.")

        end_time = time.time()
        duration_sec = end_time - start_time
        duration_str = str(datetime.timedelta(seconds=duration_sec))
        # التحقق من وجود results قبل محاولة الوصول إلى save_dir
        if results and hasattr(results, 'save_dir'):
             print(f"\n=== Training Finished ===\nBest model path: {results.save_dir / 'weights' / 'best.pt'}")
        else:
             print("\n=== Training Finished (Results object might be empty or missing save_dir) ===")
        print(f"Total duration: {duration_str}")

    except Exception as e:
        print(f"\nAn error occurred during training: {e}")
        import traceback
        traceback.print_exc() # لطباعة تتبع الخطأ الكامل

if __name__ == "__main__":
    train_pitch_pose_model()

Training started at: 2025-04-09 14:02:56.972639
Model yolov8m-pose.pt loaded successfully.
Starting model training...
New https://pypi.org/project/ultralytics/8.3.105 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.103  Python-3.10.9 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4060, 8188MiB)
engine\trainer: task=pose, mode=train, model=yolov8m-pose.pt, data=C:\Users\abdul\Desktop\YoloV8_New_Attempt\data.yaml, epochs=250, time=None, patience=50, batch=8, imgsz=960, save=True, save_period=-1, cache=ram, device=0, workers=0, project=Pitch_Pose_Project_ColorAugOnly, name=pose_e250_img960_clrAug_1744196576, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=3

train: Scanning C:\Users\abdul\Desktop\YoloV8_New_Attempt\datasets\football_pitch\labels\train.cache... 435 images, 0 backgrounds, 0 corrupt: 100%|██████████| 435/435 [00:00<?, ?it/s]


WARNING  cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (0.6GB RAM): 100%|██████████| 435/435 [00:03<00:00, 126.12it/s]
val: Scanning C:\Users\abdul\Desktop\YoloV8_New_Attempt\datasets\football_pitch\labels\val.cache... 82 images, 0 backgrounds, 0 corrupt: 100%|██████████| 82/82 [00:00<?, ?it/s]


WARNING  cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


...

     91/250      7.88G     0.1532      1.672     0.2003     0.1778     0.7613          3        960: 100%|██████████| 55/55 [02:32<00:00,  2.78s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:07<00:00,  1.18s/it]

                   all         82         82      0.834      0.829      0.894      0.725      0.642      0.659      0.585      0.406



      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


     92/250      7.88G     0.1568      1.266     0.2047     0.1851     0.7493          8        960:   9%|▉         | 5/55 [01:05<10:56, 13.12s/it]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

# مسارات الصور (غيّر المسارات حسب موقعك الفعلي إذا اختلفت)
paths = {
    "BoxP": r"BoxF1_curve.png",
    "BoxF1": r"BoxP_curve.png",
    "BoxPR": r"BoxPR_curve.png",
    "BoxR": r"BoxR_curve.png",
    "Confusion Matrix": r"confusion_matrix.png",
    "Confusion Matrix (Normalized)": r"confusion_matrix_normalized.png",
    "PoseF1": r"PoseF1_curve.png",
    "PoseP": r"PosePR_curve.png",
    "PosePR": r"PoseR_curve.png"
}

fig, axs = plt.subplots(3, 3, figsize=(18, 12))
fig.suptitle("YOLOv8 Pose Model Evaluation Results", fontsize=20)

for i, (title, path) in enumerate(paths.items()):
    row = i // 3
    col = i % 3
    if os.path.exists(path):
        img = mpimg.imread(path)
        axs[row, col].imshow(img)
        axs[row, col].axis('off')
        axs[row, col].set_title(title, fontsize=14)
    else:
        axs[row, col].axis('off')
        axs[row, col].set_title(f"{title}\n[Missing]", fontsize=14, color='red')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig("combined_results.png", dpi=300)
print("combined_results.png")

✅ تم حفظ الصورة بنجاح: combined_results.png


In [ ]:
import cv2
import os
# import random
import numpy as np
from ultralytics import YOLO
import yaml # لقراءة ملف data.yaml
import time
import datetime
import collections
import traceback # لطباعة الأخطاء بشكل أفضل

# --- الإعدادات الرئيسية ---

# !!!!!!!!!! تحديث ضروري !!!!!!!!!!
# 1. ضع المسار الصحيح لنموذجك 'best.pt'
MODEL_PATH = r'C:\Users\abdul\Desktop\YoloV8_New_Attempt\datasets\labels\validation\Pitch_Pose_Project_ColorAugOnly\pose_e250_img960_clrAug_1744196576\weights\best.pt' # <--- تأكد منه

# 2. تأكد من صحة مسار ملف data.yaml
DATA_YAML_PATH = r'C:\Users\abdul\Desktop\YoloV8_New_Attempt\data.yaml' # <--- تأكد منه

# 3. (مهم للعرض) عرف قائمة SKELETON بناءً على فهارس نقاطك الـ 14
SKELETON = [
    # أضف الوصلات هنا، مثال: [[0, 1], [1, 3], ...]
] # <--- عدّل هذه القائمة

# --- إعدادات أخرى ---
OUTPUT_FOLDER = 'evaluation_visuals_TEST_set_final' # اسم مجلد جديد للمخرجات
CONF_THRESHOLD = 0.5
KPT_CONF_THRESHOLD = 0.5
TRAINING_IMG_SIZE = 960 # حجم الصور المستخدم في التدريب

# --- تحميل معلومات المسار والنقاط من data.yaml ---
test_image_path = None # المسار لمجلد صور الاختبار
keypoint_labels = []
num_keypoints = 14
data_yaml_dir = None
data_config = None

print(f"Attempting to load data config from: {DATA_YAML_PATH}")
try:
    if not os.path.exists(DATA_YAML_PATH):
        print(f"ERROR: data.yaml file not found at '{DATA_YAML_PATH}'")
    else:
        data_yaml_dir = os.path.dirname(DATA_YAML_PATH)
        print(f"YAML directory: {data_yaml_dir}")
        with open(DATA_YAML_PATH, 'r', encoding='utf-8') as f:
            data_config = yaml.safe_load(f)

        if data_config:
            # --- استخراج مسار الاختبار (test) ---
            if 'test' in data_config:
                 test_relative_path = data_config['test']
                 print(f"Found 'test' path in YAML: {test_relative_path}")
                 potential_path = os.path.join(data_yaml_dir, test_relative_path)
                 print(f"Checking relative path: {potential_path}")
                 if os.path.isdir(potential_path):
                      test_image_path = os.path.abspath(potential_path)
                      print(f"Using TEST image path (relative): {test_image_path}")
                 elif os.path.isdir(test_relative_path):
                      test_image_path = os.path.abspath(test_relative_path)
                      print(f"Using TEST image path (absolute): {test_image_path}")
                 else:
                      print(f"ERROR: Test path specified in YAML ('{test_relative_path}') not found relative to YAML ('{potential_path}') or as absolute path.")
            else:
                print(f"ERROR: 'test' key not found in {DATA_YAML_PATH}")

            # --- تحميل معلومات النقاط الرئيسية ---
            if 'kpt_shape' in data_config:
                 num_keypoints = data_config.get('kpt_shape', [14])[0]
                 keypoint_labels = [str(i) for i in range(num_keypoints)]
                 print(f"Loaded {num_keypoints} keypoint indices as labels.")
            else: print(f"Warning: 'kpt_shape' not found. Using default {num_keypoints}."); keypoint_labels = [str(i) for i in range(num_keypoints)]
        else: print("ERROR: Failed to load data from YAML file.")

except Exception as e: print(f"Error loading or parsing {DATA_YAML_PATH}: {e}")

# --- توليد ألوان مميزة للنقاط ---
rng = np.random.default_rng(1); keypoint_colors = rng.uniform(50, 255, size=(len(keypoint_labels), 3)).astype(np.uint8)

# ========== دالة حساب التقاطع فوق الاتحاد (IoU) ==========
def calculate_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0]); yA = max(boxA[1], boxB[1]); xB = min(boxA[2], boxB[2]); yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1]); boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    unionArea = float(boxAArea + boxBArea - interArea); iou = interArea / unionArea if unionArea > 0 else 0
    return iou

# ========== الدالة الرئيسية للتقييم والعرض ==========
def evaluate_and_visualize(model_path, data_yaml, test_img_path, output_folder, imgsz_eval, conf_thresh=0.5, kpt_thresh=0.5):
    """يحمل النموذج، يقيمه على مجموعة الاختبار، ويرسم التنبؤات على مجموعة الاختبار."""

    print(f"\nLoading model from: {model_path}")
    try: model = YOLO(model_path); print("Model loaded."); print(f"Using image size: {imgsz_eval}")
    except Exception as e: print(f"Error loading model: {e}"); return

    # --- 1. التقييم الرقمي (Quantitative Evaluation) على مجموعة الاختبار ---
    print("\n--- Running Quantitative Validation on TEST set ---")
    try:
        # ***** استخدام split='test' هنا *****
        metrics = model.val(data=data_yaml, imgsz=imgsz_eval, conf=conf_thresh, split='test')
        print("\nTest Set Metrics Summary:")
        print(f"Box mAP50 (Test): {metrics.box.map50:.4f}"); print(f"Box mAP50-95 (Test): {metrics.box.map:.4f}")
        print(f"Pose mAP50 (OKS? Test): {metrics.pose.map50:.4f} (May be misleading)"); print(f"Pose mAP50-95 (OKS? Test): {metrics.pose.map:.4f} (May be misleading)")
    except KeyError as e: print(f"Error running validation on test set: Missing key {e}. YAML missing 'test' path?")
    except Exception as e: print(f"Error running validation on test set: {e}.")

    # --- 2. التقييم البصري (Visual Evaluation) على مجموعة الاختبار ---
    # التأكد من أن مسار صور الاختبار صحيح وموجود
    if not test_img_path or not os.path.isdir(test_img_path):
         print(f"\nERROR: Skipping visual evaluation because test image path is invalid or not found: '{test_img_path}'")
         print("Please ensure the 'test:' path in your data.yaml points to the correct image directory.")
         return

    print(f"\n--- Running Visual Evaluation using TEST images from: {test_img_path} ---")
    os.makedirs(output_folder, exist_ok=True)
    print(f"Visual results will be saved to: {os.path.abspath(output_folder)}")

    try:
        # ***** تغيير هنا: استخدام source=test_img_path وإزالة data و split *****
        print(f"Starting prediction using source='{test_img_path}'...")
        results_generator = model.predict(source=test_img_path, conf=conf_thresh, imgsz=imgsz_eval, stream=True, save=False)

        processed_count = 0
        total_images_estimated = '?'
        print(f"Iterating through prediction results...")

        for result in results_generator:
            processed_count += 1
            image_path = getattr(result, 'path', f'image_{processed_count}')
            filename = os.path.basename(image_path)
            if processed_count % 5 == 0 or processed_count <= 5: # طباعة كل 5 صور أو أول 5
                 print(f"Processing ({processed_count}): {filename}")

            # --- كود الرسم والحفظ (لا تغيير هنا) ---
            try:
                img = result.orig_img if hasattr(result, 'orig_img') and result.orig_img is not None else cv2.imread(image_path)
                if img is None: print(f"  Warning: Could not get image for {filename}"); continue
                annotated_img = img.copy()
                if not hasattr(result, 'boxes') or result.boxes is None or len(result.boxes) == 0:
                    output_path = os.path.join(output_folder, f"annotated_{filename}"); cv2.imwrite(output_path, annotated_img); continue # حفظ الصورة الأصلية

                box = result.boxes[0]; class_id = int(box.cls[0]); class_name = model.names[class_id] if class_id in model.names else f"ID_{class_id}"
                x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy()); cv2.rectangle(annotated_img, (x1, y1), (x2, y2), (0, 255, 0), 2); cv2.putText(annotated_img, f"{class_name} {box.conf[0]:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                if hasattr(result, 'keypoints') and result.keypoints is not None and result.keypoints.shape[1] > 0:
                    keypoints = result.keypoints; points_xy = keypoints.xy[0].cpu().numpy() if keypoints.xy is not None else None; points_conf = None
                    if hasattr(keypoints, 'conf') and keypoints.conf is not None and keypoints.conf.shape[1] > 0: points_conf = keypoints.conf[0].cpu().numpy()
                    if points_xy is not None:
                        num_kpts_detected = points_xy.shape[0]
                        for i in range(num_kpts_detected):
                           kpt_visible = True
                           if points_conf is not None and i < len(points_conf):
                               if points_conf[i] < kpt_thresh: kpt_visible = False
                           if kpt_visible:
                               px, py = int(points_xy[i, 0]), int(points_xy[i, 1])
                               if 0 <= px < annotated_img.shape[1] and 0 <= py < annotated_img.shape[0]:
                                   color = tuple(map(int, keypoint_colors[i % len(keypoint_colors)])); label = keypoint_labels[i] if i < len(keypoint_labels) else str(i)
                                   cv2.circle(annotated_img, (px, py), 5, color, -1); cv2.circle(annotated_img, (px, py), 7, (255, 255, 255), 1); cv2.putText(annotated_img, label, (px + 6, py - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)
                        if SKELETON:
                           for ki, kj in SKELETON:
                               pt1_visible = False; pt2_visible = False
                               if ki < num_kpts_detected: pt1_visible = (points_conf is None or (ki < len(points_conf) and points_conf[ki] >= kpt_thresh))
                               if kj < num_kpts_detected: pt2_visible = (points_conf is None or (kj < len(points_conf) and points_conf[kj] >= kpt_thresh))
                               if pt1_visible and pt2_visible:
                                    pt1 = tuple(map(int, points_xy[ki])); pt2 = tuple(map(int, points_xy[kj]))
                                    if 0 <= pt1[0] < annotated_img.shape[1] and 0 <= pt1[1] < annotated_img.shape[0] and 0 <= pt2[0] < annotated_img.shape[1] and 0 <= pt2[1] < annotated_img.shape[0]:
                                         color = tuple(map(int, keypoint_colors[ki % len(keypoint_colors)])); cv2.line(annotated_img, pt1, pt2, color, 2)

                output_path = os.path.join(output_folder, f"annotated_{filename}")
                cv2.imwrite(output_path, annotated_img)

            except KeyboardInterrupt: print("Visualization interrupted."); raise
            except Exception as e: print(f"  Error processing {filename}: {e}"); traceback.print_exc()

        print(f"\nFinished processing {processed_count} test images.")
        print(f"Annotated test images saved in: {os.path.abspath(output_folder)}")

    except KeyboardInterrupt: print("\nPrediction loop interrupted.")
    except Exception as e: print(f"An error occurred: {e}"); traceback.print_exc()


if __name__ == "__main__":
    # التحقق من تحميل مسار الاختبار قبل المتابعة
    if test_image_path and os.path.isdir(test_image_path):
        start_vis_time = time.time()
        evaluate_and_visualize(
            model_path=MODEL_PATH,
            data_yaml=DATA_YAML_PATH, # لا يزال مطلوبًا لـ .val()
            test_img_path=test_image_path, # <-- تمرير المسار الصحيح
            output_folder=OUTPUT_FOLDER,
            imgsz_eval=TRAINING_IMG_SIZE, # <-- استخدام حجم الصورة الصحيح
            conf_thresh=CONF_THRESHOLD,
            kpt_thresh=KPT_CONF_THRESHOLD
        )
        end_vis_time = time.time(); duration_vis_sec = end_vis_time - start_vis_time; duration_vis_str = str(datetime.timedelta(seconds=duration_vis_sec))
        print(f"\nEvaluation & Visualization task duration: {duration_vis_str}")
    else:
        print("\nCould not run evaluation/visualization because the 'test' image path was not determined or found.")
        print(f"Please check the 'test:' path in '{DATA_YAML_PATH}' and ensure the directory exists.")

Attempting to load data config from: C:\Users\abdul\Desktop\YoloV8_New_Attempt\data.yaml
YAML directory: C:\Users\abdul\Desktop\YoloV8_New_Attempt
Found 'test' path in YAML: C:\Users\abdul\Desktop\YoloV8_New_Attempt\datasets\football_pitch\images\test
Checking relative path: C:\Users\abdul\Desktop\YoloV8_New_Attempt\datasets\football_pitch\images\test
Using TEST image path (relative): C:\Users\abdul\Desktop\YoloV8_New_Attempt\datasets\football_pitch\images\test
Loaded 14 keypoint indices as labels.

Loading model from: C:\Users\abdul\Desktop\YoloV8_New_Attempt\datasets\labels\validation\Pitch_Pose_Project_ColorAugOnly\pose_e250_img960_clrAug_1744196576\weights\best.pt
Model loaded.
Using image size: 960

--- Running Quantitative Validation on TEST set ---
Ultralytics 8.3.103  Python-3.10.9 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4060, 8188MiB)
YOLOv8m-pose summary (fused): 101 layers, 26,406,673 parameters, 0 gradients, 80.9 GFLOPs


val: Scanning C:\Users\abdul\Desktop\YoloV8_New_Attempt\datasets\football_pitch\labels\test.cache... 86 images, 0 backgrounds, 0 corrupt: 100%|██████████| 86/86 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:43<00:00,  7.32s/it]


                   all         86         86      0.885       0.93      0.955      0.805      0.808      0.779       0.83      0.563
Speed: 3.4ms preprocess, 421.8ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs\pose\val6

Test Set Metrics Summary:
Box mAP50 (Test): 0.9548
Box mAP50-95 (Test): 0.8050
Pose mAP50 (OKS? Test): 0.8299 (May be misleading)
Pose mAP50-95 (OKS? Test): 0.5627 (May be misleading)

--- Running Visual Evaluation using TEST images from: C:\Users\abdul\Desktop\YoloV8_New_Attempt\datasets\football_pitch\images\test ---
Visual results will be saved to: c:\Users\abdul\Desktop\YoloV8_New_Attempt\datasets\labels\validation\evaluation_visuals_TEST_set_final
Starting prediction using source='C:\Users\abdul\Desktop\YoloV8_New_Attempt\datasets\football_pitch\images\test'...
Iterating through prediction results...

image 1/86 C:\Users\abdul\Desktop\YoloV8_New_Attempt\datasets\football_pitch\images\test\frame_000025.PNG: 544x960 1 pitch_layout, 47.7m

In [ ]:
import cv2
import os
import numpy as np
from ultralytics import YOLO

# ------------------ إعدادات أساسية ------------------
MODEL_PATH = r"best.pt"  # مسار نموذجك (best.pt)
INPUT_VIDEO = r"Untitled video - Made with Clipchamp (6).mp4"                       # الفيديو الأصلي
OUTPUT_VIDEO = "annotated_output.avi"                             # اسم الفيديو الناتج
CONF_THRESHOLD = 0.5
KPT_THRESHOLD = 0.5
IMG_SIZE = 960

# ترتيب النقاط بالضبط
KEYPOINTS_ORDER = [
    "Corner_Top_Left", "Line_Top", "Corner_Top_Right", "18_Top_Left",
    "18_Bottom_Left", "Penalty_Spot_Left", "Circle_Top", "Circle_Bottom",
    "Penalty_Spot_Right", "18_Top_Right", "18_Bottom_Right", "Corner_Bottom_Left",
    "Line_Bottom", "Corner_Bottom_Right"
]
num_keypoints = len(KEYPOINTS_ORDER)

# وصلات (SKELETON) بين النقاط
SKELETON = [
    [0, 1], [1, 2],
    [11, 12], [12, 13],
    [0, 11], [2, 13],
    [1, 12],
    [3, 4],
    [9, 10],
    [6, 7],
    [5, 8]
]

# توليد ألوان النقاط بشكل عشوائي
rng = np.random.default_rng(123)
keypoint_colors = rng.uniform(50, 255, size=(num_keypoints, 3)).astype(np.uint8)

def run_pose_on_video():
    # 1) حمل النموذج
    model = YOLO(MODEL_PATH)
    print(f"Model loaded from: {MODEL_PATH}")

    # 2) افتح الفيديو للقراءة
    cap = cv2.VideoCapture(INPUT_VIDEO)
    if not cap.isOpened():
        print(f"ERROR: cannot open video {INPUT_VIDEO}")
        return

    # خصائص الفيديو (FPS, W, H)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # 3) افتح فيديو للكتابة
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps, (width, height))

    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1

        # استدعاء النموذج على الإطار
        results = model.predict(frame, conf=CONF_THRESHOLD, imgsz=IMG_SIZE, stream=False)
        # غالبًا results = [Result(...)]

        if len(results) > 0:
            r = results[0]

            # ---- [إخفاء الصندوق boundary] => لا نرسم box.xyxy ----
            # => نحذف الكود الذي يرسم الـ rectangle والاسم (class_name)

            # ارسم النقاط
            if r.keypoints is not None and r.keypoints.shape[1] > 0:
                kpts = r.keypoints
                if kpts.xy is not None:
                    points_xy = kpts.xy[0].cpu().numpy()  # [N,2]
                else:
                    points_xy = None

                points_conf = None
                if hasattr(kpts, 'conf') and kpts.conf is not None:
                    points_conf = kpts.conf[0].cpu().numpy()  # [N]

                if points_xy is not None:
                    for i in range(points_xy.shape[0]):
                        px, py = int(points_xy[i, 0]), int(points_xy[i, 1])
                        c_kpt = 1.0
                        if points_conf is not None and i < len(points_conf):
                            c_kpt = points_conf[i]
                        # إذا ثقة النقطة >= KPT_THRESHOLD
                        if c_kpt >= KPT_THRESHOLD:
                            color = tuple(map(int, keypoint_colors[i]))
                            label = KEYPOINTS_ORDER[i]  # أو فقط str(i)

                            # تكبير دائرة النقطة + زيادة الخط
                            cv2.circle(frame, (px, py), 8, color, -1)  # دائرة مملوءة
                            # إضافة حدود بيضاء لزيادة الوضوح
                            cv2.circle(frame, (px, py), 10, (255, 255, 255), 2)

                            # تكبير النص قليلاً + لون أسود للخلفية (shadow)
                            cv2.putText(frame, label, (px + 10, py),
                                        cv2.FONT_HERSHEY_SIMPLEX,
                                        0.6,             # الحجم
                                        (0, 0, 0),       # اللون الأسود كمصغر خلفية
                                        4,               # السماكة =4 (للخلفية)
                                        cv2.LINE_AA)
                            cv2.putText(frame, label, (px + 10, py),
                                        cv2.FONT_HERSHEY_SIMPLEX,
                                        0.6,             # الحجم
                                        (255, 255, 255), # لون أبيض فوق
                                        1,
                                        cv2.LINE_AA)

                    # ارسم وصلات الملعب
                    for (a,b) in SKELETON:
                        if a < points_xy.shape[0] and b < points_xy.shape[0]:
                            c1 = 1.0 if points_conf is None else points_conf[a]
                            c2 = 1.0 if points_conf is None else points_conf[b]
                            if c1 >= KPT_THRESHOLD and c2 >= KPT_THRESHOLD:
                                pt1 = tuple(map(int, points_xy[a]))
                                pt2 = tuple(map(int, points_xy[b]))
                                color = tuple(map(int, keypoint_colors[a]))
                                cv2.line(frame, pt1, pt2, color, 3)  # زيادة السماكة = 3

        out.write(frame)

    cap.release()
    out.release()
    print(f"✅ Done! Video saved as: {OUTPUT_VIDEO}")

if __name__ == "__main__":
    run_pose_on_video()


Model loaded from: C:\Users\abdul\Desktop\YoloV8_New_Attempt\datasets\labels\validation\Pitch_Pose_Project_ColorAugOnly\pose_e250_img960_clrAug_1744196576\weights\best.pt

0: 544x960 1 pitch_layout, 280.8ms
Speed: 33.6ms preprocess, 280.8ms inference, 97.3ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 1 pitch_layout, 187.3ms
Speed: 10.0ms preprocess, 187.3ms inference, 2.5ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 1 pitch_layout, 182.7ms
Speed: 6.8ms preprocess, 182.7ms inference, 2.4ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 1 pitch_layout, 183.9ms
Speed: 7.1ms preprocess, 183.9ms inference, 2.8ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 1 pitch_layout, 206.6ms
Speed: 7.2ms preprocess, 206.6ms inference, 4.6ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 1 pitch_layout, 185.5ms
Speed: 7.0ms preprocess, 185.5ms inference, 4.5ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 1 pitch_la